In [ ]:
!pip install twstock
!pip install pandas_ta


In [ ]:
import twstock
import pandas as pd
import pandas_ta as ta
from tqdm import tqdm
# reference https://github.com/mlouielu/twstock

#stock_ids = stock_num_list[:1]
stock_ids = ['2330'] # stock id
start_year = 2010
start_month = 1

for sid in tqdm(stock_ids):
  stock = twstock.Stock(sid)
  target_price = stock.fetch_from(start_year, start_month)  #取用yyyy/mm至今每天的交易資料
  #title
  name_attribute = [
      'Date', 'Capacity', 'Turnover', 'Open', 'High', 'Low', 'Close', 'Change',
      'Transaction'
  ]
  #transform list to data sheet
  df = pd.DataFrame(columns=name_attribute, data=target_price)
  df = df[df['Capacity'] != 0].reset_index(drop=True)

  df['MA5'] = df['Close'].rolling(window=5).mean()    #五日均價
  df['MA10'] = df['Close'].rolling(window=10).mean()    #十日均價
  df['MA5_Capacity'] = df['Capacity'].rolling(window=5).mean() #五日均量
  df['BR5'] = (df['Close'] - df['MA5']) / df['MA5'] * 100 #五日乖離值
  df['BR10'] = (df['Close'] - df['MA10']) / df['MA10'] * 100 #十日乖離值
  df['High-Low'] = df['High'] - df['Low']     #最高-最低
  df['Open-Close'] = df['Open'] - df['Close']   #開盤-收盤

  # 指數移動平均
  df['EMA5'] = df['Close'].ewm(span=5, adjust=False).mean()
  df['EMA10'] = df['Close'].ewm(span=10, adjust=False).mean()

  # KD 指標
  kd = ta.stoch(df['High'], df['Low'], df['Close'])
  df['K'] = kd['STOCHk_14_3_3']
  df['D'] = kd['STOCHd_14_3_3']

  # MACD
  macd = ta.macd(df['Close'])
  df['MACD'] = macd['MACD_12_26_9']
  df['MACD_signal'] = macd['MACDs_12_26_9']
  df['MACD_hist'] = macd['MACDh_12_26_9']

  # RSI
  df['RSI14'] = ta.rsi(df['Close'], length=14)

  # 標準差
  df['STD20'] = df['Close'].rolling(20).std()

  # ADX
  adx = ta.adx(df['High'], df['Low'], df['Close'])
  df['ADX'] = adx['ADX_14']

  df.to_csv(f"{sid}.csv", index=False)

100%|██████████| 1/1 [00:27<00:00, 27.48s/it]
